In [ ]:
!pip install numpy mido pygame --quiet

In [ ]:
import numpy
print("NumPy è stato importato correttamente!")

In [ ]:
# Installazione delle librerie necessarie
!pip install numpy
!pip install mido
!pip install pygame

In [ ]:
import numpy as np
import random
from mido import Message, MidiFile, MidiTrack, tempo2bpm
from typing import List, Dict, Any, Tuple
import time
import mido
from mido import Message, MidiFile, MidiTrack

In [ ]:
# MIDI Note numbers per la scala di Do maggiore (C4 a B4)
PITCH_RANGE = list(range(60, 72))  # Da C4 (60) a B4 (71)
MAX_VELOCITY = 100                 # Intensità del suono
NOTE_DURATION = 0.5                # Durata di base in secondi

In [ ]:
# Progressione di accordi (I-IV-V-I in C maggiore)
CHORD_PROGRESSION = [60, 65, 67, 60] * 2  # C, F, G, C ripetuto

# Dizionario degli accordi validi
ACCORDI_VALIDI = {
    60: [60, 64, 67, 72],  # C Maj (C, E, G, C)
    65: [65, 69, 72, 77],  # F Maj (F, A, C, F)
    67: [67, 71, 74, 79],  # G Maj (G, B, D, G)
}


In [ ]:
POPULATION_SIZE = 100                     # Numero di individui per generazione
MELODY_LENGTH = len(CHORD_PROGRESSION) * 4  # Lunghezza della melodia (32 note)
GENERATIONS = 50                          # Numero di generazioni
MUTATION_RATE = 0.05                      # Tasso di mutazione
CROSSOVER_RATE = 0.7                      # Tasso di crossover

In [ ]:
# Genoma: un array di tuple, dove ogni tupla è (pitch, velocity) per una nota.
# Non ottimizziamo la durata (duration) per mantenere la semplicità.
# L'individuo sarà un dizionario simile a quelli usati nei tuoi notebook.
# Esempio di genoma: [(60, 90), (62, 85), (64, 90), ...]

def create_random_genoma(length: int) -> List[Tuple[int, int]]:
    """Crea un genoma (melodia) casuale della lunghezza specificata."""
    genoma = []
    for _ in range(length):
        pitch = random.choice(PITCH_RANGE)
        velocity = random.randint(50, MAX_VELOCITY)
        genoma.append((pitch, velocity))
    return genoma

def initialize_population(size: int) -> List[Dict[str, Any]]:
    """Inizializza la popolazione iniziale di genomi."""
    population = []
    for _ in range(size):
        population.append({
            'genoma': create_random_genoma(MELODY_LENGTH),
            'fitness': 0.0 # Verrà calcolata
        })
    return population

In [ ]:
def calculate_fitness(individual: Dict[str, Any]) -> float:
    """
    Calcola il punteggio di fitness di un individuo (melodia).
    L'obiettivo è MASSIMIZZARE questo punteggio (minimizzare il "malus").
    """
    genoma = individual['genoma']
    fitness_score = 0.0

    # 1. Armonia (Coerenza con gli Accordi Target) - BONUS GRANDE
    # Per ogni nota, controlliamo se è una nota valida dell'accordo in quel punto
    for i, (pitch, _) in enumerate(genoma):
        chord_index = i // 4 # Determina l'accordo (cambia ogni 4 note)
        target_chord_root = CHORD_PROGRESSION[chord_index]

        # Mappa l'accordo target alle note valide
        valid_notes = ACCORDI_VALIDI.get(target_chord_root, [])

        # Applica il concetto di 'Coerenza Armonica':
        if pitch in valid_notes:
            fitness_score += 10.0  # Bonus per nota nell'accordo
        else:
            fitness_score -= 5.0   # Malus per nota dissonante

    # 2. Conduzione Vocale (Flow/Fluidità) - MALUS PICCOLO
    # Penalizza i salti melodici grandi (poco naturali)
    for i in range(1, MELODY_LENGTH):
        pitch_prev = genoma[i-1][0]
        pitch_curr = genoma[i][0]
        pitch_difference = abs(pitch_curr - pitch_prev)

        # Applica il concetto di 'Conduzione Vocale': salti di più di 5 semitoni sono penalizzati.
        if pitch_difference > 5:
            fitness_score -= (pitch_difference - 5) * 1.5 # Penalità proporzionale

    # 3. Varianza (Per evitare melodie troppo monotone) - BONUS PICCOLO
    # Bonus se la melodia usa un range ragionevole di note
    pitches = [p for p, v in genoma]
    pitch_variance = np.var(pitches)
    fitness_score += pitch_variance * 0.5 # Premia la varianza

    return fitness_score

def fitness(population: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """Calcola la fitness per l'intera popolazione e ordina."""
    for individual in population:
        individual['fitness'] = calculate_fitness(individual)

    # Ordina la popolazione in base alla fitness (dal migliore al peggiore)
    population.sort(key=lambda x: x['fitness'], reverse=True)
    return population

In [ ]:
def selection(population: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Selezione basata su un approccio ibrido: elitismo e selezione casuale pesata.
    Mantiene il 10% dei migliori e seleziona il resto da un pool pesato.
    """
    num_elite = int(POPULATION_SIZE * 0.1)
    elite = population[:num_elite]

    # Candidati da cui selezionare i genitori
    candidates = population[num_elite:]
    total_fitness = sum(i['fitness'] for i in candidates)

    if total_fitness <= 0:
        weights = [1] * len(candidates)
    else:
        weights = [i['fitness'] / total_fitness for i in candidates]

    parents = random.choices(candidates, weights=weights, k=POPULATION_SIZE - num_elite)
    return elite + parents

def crossover(population: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Applica il Single Point Crossover (come nel file 02-- Algoritmi genetici.pdf).
    """
    new_population = []

    # Manteniamo l'individuo migliore non modificato (Elitismo)
    new_population.append(population[0])

    for i in range(1, POPULATION_SIZE // 2):
        parent1 = population[i * 2]
        parent2 = population[i * 2 + 1]

        if random.random() < CROSSOVER_RATE:
            # Scegli un punto di crossover casuale
            point = random.randint(1, MELODY_LENGTH - 1)

            # Crea due nuovi figli
            child1_genoma = parent1['genoma'][:point] + parent2['genoma'][point:]
            child2_genoma = parent2['genoma'][:point] + parent1['genoma'][point:]

            new_population.append({'genoma': child1_genoma, 'fitness': 0.0})
            new_population.append({'genoma': child2_genoma, 'fitness': 0.0})
        else:
            new_population.append(parent1)
            new_population.append(parent2)

    # Assicura che la dimensione sia mantenuta
    return new_population[:POPULATION_SIZE]


def mutation(population: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Applica la mutazione: cambia casualmente il pitch di alcune note.
    """
    # Evitiamo di mutare il genoma migliore (Elitismo)
    for individual in population[1:]:
        genoma = individual['genoma']
        for i in range(MELODY_LENGTH):
            if random.random() < MUTATION_RATE:
                # Esegui la mutazione:
                # 1. Scegli un nuovo pitch casuale
                new_pitch = random.choice(PITCH_RANGE)
                # 2. Aggiorna la tupla nel genoma
                genoma[i] = (new_pitch, genoma[i][1]) # Manteniamo la velocity

    return population

In [ ]:
def create_midi_file(genoma: List[Tuple[int, int]], filename: str = "melodia_evoluta.mid"):
    """Converte il genoma (melodia) nel file MIDI riproducibile."""
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    tempo = mido.bpm2tempo(120)  # 120 BPM
    track.append(mido.MetaMessage('set_tempo', tempo=tempo))
    track.append(mido.Message('program_change', program=1, time=0))  # Piano

    # Converte la durata in secondi in 'ticks' (unità MIDI)
    ticks_per_beat = mid.ticks_per_beat
    note_time = int(NOTE_DURATION * ticks_per_beat)

    # Aggiunge le note
    for pitch, velocity in genoma:
        track.append(Message('note_on', note=pitch, velocity=velocity, time=0))
        track.append(Message('note_off', note=pitch, velocity=velocity, time=note_time))

    mid.save(filename)
    print(f"File MIDI salvato come: {filename}")


def run_genetic_algorithm():
    """Funzione principale per eseguire l'evoluzione."""

    # Inizializza la popolazione e il genoma migliore
    popolazione = initialize_population(POPULATION_SIZE)
    popolazione = fitness(popolazione)

    # ✅ Salva la melodia iniziale prima che inizi l'evoluzione
    create_midi_file(popolazione[0]['genoma'], filename="melodia_iniziale.mid")

    best_individual = popolazione[0]

    print(f"Target melodia: {MELODY_LENGTH} note, {GENERATIONS} generazioni.")
    print("---------------------------------------------------------")

    fitness_history = []
    # Ciclo Evolutivo
    for generation in range(GENERATIONS):

        # 1. Selezione (per il pool di riproduzione)
        popolazione = selection(popolazione)

        # 2. Crossover (Crea nuovi individui)
        popolazione = crossover(popolazione)

        # 3. Mutazione (Introduce casualità)
        popolazione = mutation(popolazione)

        # 4. Fitness (Valuta la nuova generazione)
        popolazione = fitness(popolazione)

        fitness_history.append(popolazione[0]['fitness'])

        # 5. Aggiornamento del Migliore (Elitismo)
        if popolazione[0]['fitness'] > best_individual['fitness']:
            best_individual = popolazione[0]

        # Output dello stato
        if (generation + 1) % 5 == 0 or generation == 0:
            print(f"Generazione {generation+1:2d}/{GENERATIONS} - Fitness Max: {best_individual['fitness']:.2f} (Pop. attuale: {popolazione[0]['fitness']:.2f})")

    print("---------------------------------------------------------")
    print(f"Evoluzione completata. Fitness finale migliore: {best_individual['fitness']:.2f}")

    # Crea il file MIDI con la soluzione ottimale
    create_midi_file(best_individual['genoma'])

    # Visualizza l'evoluzione della fitness
    !pip install matplotlib
    import matplotlib.pyplot as plt
    plt.plot(fitness_history)
    plt.xlabel("Generazione")
    plt.ylabel("Fitness massima")
    plt.title("Evoluzione della fitness")
    plt.grid(True)
    plt.show()

    # ✅ Confronto tra melodia iniziale e finale
    import numpy as np

    # 1. Quante note erano armoniche
    initial_harmony_score = calculate_fitness(popolazione[0]['genoma'])[0]
    final_harmony_score = calculate_fitness(best_individual['genoma'])[0]
    print(f"Note armoniche - Iniziale: {initial_harmony_score:.2f}, Finale: {final_harmony_score:.2f}")

    # 2. Varianza dei pitch
    initial_pitches = [p for p, v in popolazione[0]['genoma']]
    final_pitches = [p for p, v in best_individual['genoma']]
    print(f"Varianza pitch - Iniziale: {np.var(initial_pitches):.2f}, Finale: {np.var(final_pitches):.2f}")

    # 3. Fluidità (salti melodici)
    initial_jumps = [abs(initial_pitches[i+1] - initial_pitches[i]) for i in range(len(initial_pitches)-1)]
    final_jumps = [abs(final_pitches[i+1] - final_pitches[i]) for i in range(len(final_pitches)-1)]
    print(f"Media salti - Iniziale: {np.mean(initial_jumps):.2f}, Finale: {np.mean(final_jumps):.2f}")


    return best_individual

# Esecuzione del progetto!
melodia_ottimale = run_genetic_algorithm()